In [1]:
import numpy as np
import pandas as pd
import random
import math,time,sys
from matplotlib import pyplot
from datetime import datetime
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#==================================================================
def sigmoid1(gamma):     #convert to probability
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid1i(gamma):     #convert to probability
	gamma = -gamma
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid2(gamma):
	gamma /= 2
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid3(gamma):
	gamma /= 3
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid4(gamma):
	gamma *= 2
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))


def Vfunction1(gamma):
	return abs(np.tanh(gamma))

def Vfunction2(gamma):
	val = (math.pi)**(0.5)
	val /= 2
	val *= gamma
	val = math.erf(val)
	return abs(val)

def Vfunction3(gamma):
	val = 1 + gamma*gamma
	val = math.sqrt(val)
	val = gamma/val
	return abs(val)

def Vfunction4(gamma):
	val=(math.pi/2)*gamma
	val=np.arctan(val)
	val=(2/math.pi)*val
	return abs(val)

def initialize(popSize,dim):
	population=np.zeros((popSize,dim))
	minn = 1
	maxx = math.floor(0.8*dim)
	if maxx<minn:
		minn = maxx

	for i in range(popSize):
		random.seed(i**3 + 10 + time.time() ) 
		no = random.randint(minn,maxx)
		if no == 0:
			no = 1
		random.seed(time.time()+ 100)
		pos = random.sample(range(0,dim-1),no)
		for j in pos:
			population[i][j]=1

		# print(population[i])  
	return population

def fitness(solution, trainX, testX, trainy, testy):
	cols=np.flatnonzero(solution)
	val=1
	if np.shape(cols)[0]==0:
		return val	
	clf=RandomForestClassifier(n_estimators=10)
	train_data=trainX[:,cols]
	test_data=testX[:,cols]
	clf.fit(train_data,trainy)
	val=1-clf.score(test_data,testy)

	#in case of multi objective  []
	set_cnt=sum(solution)
	set_cnt=set_cnt/np.shape(solution)[0]
	val=omega*val+(1-omega)*set_cnt
	return val

def allfit(population, trainX, testX, trainy, testy):
	x=np.shape(population)[0]
	acc=np.zeros(x)
	for i in range(x):
		acc[i]=fitness(population[i],trainX,testX,trainy,testy)     
		#print(acc[i])
	return acc

def toBinary(solution,dimension,trainX,testX,trainy,testy):
	# print("continuous",solution)

	Xnew = np.zeros(np.shape(solution))
	for i in range(dimension):
		temp = sigmoid1(solution[i])

		random.seed(time.time()+i)
		if temp > 0.5: # sfunction
			Xnew[i] = 1
		else:
			Xnew[i] = 0
		# if temp > 0.5: # vfunction
		# 	Xnew[i] = 1 - solution[i]
		# else:
		# 	Xnew[i] = solution[i]

	# Xnew = np.zeros(np.shape(solution))
	# Xnew1 = np.zeros(np.shape(solution))
	# Xnew2 = np.zeros(np.shape(solution))
	# for i in range(dimension):
	# 	temp = sigmoid1(abs(solution[i]))
	# 	random.seed(time.time()+i)
	# 	r1 = random.random()
	# 	if temp > r1: # sfunction
	# 		Xnew1[i] = 1
	# 	else:
	# 		Xnew1[i] = 0

	# 	temp = sigmoid1i(abs(solution[i]))
	# 	if temp > r1: # sfunction
	# 		Xnew2[i] = 1
	# 	else:
	# 		Xnew2[i] = 0

	# fit1 = fitness(Xnew1,trainX,testX,trainy,testy)
	# fit2 = fitness(Xnew2,trainX,testX,trainy,testy)
	# fitOld =  fitness(solution,trainX,testX,trainy,testy)
	# if fit1<fitOld or fit2<fitOld:
	# 	if fit1 < fit2:
	# 		Xnew = Xnew1.copy()
	# 	else:
	# 		Xnew = Xnew2.copy()
	# return Xnew
	# # else: CROSSOVER
	# Xnew3 = Xnew1.copy()
	# Xnew4 = Xnew2.copy()
	# for i in range(dimension):
	# 	random.seed(time.time() + i)
	# 	r2 = random.random()
	# 	if r2>0.5:
	# 		tx = Xnew3[i]
	# 		Xnew3[i] = Xnew4[i]
	# 		Xnew4[i] = tx
	# fit1 = fitness(Xnew3,trainX,testX,trainy,testy)
	# fit2 = fitness(Xnew4,trainX,testX,trainy,testy)
	# if fit1<fit2:
	# 	return Xnew3
	# else:
	# 	return Xnew4
	# print("binary",Xnew)
	return Xnew


#============================================================================
def socialmimic(dataset,popSize,maxIter):

	#--------------------------------------------------------------------
	df=pd.read_csv(dataset)
	(a,b)=np.shape(df)
	#print(a,b)
	data = df.values[:,0:b-1]
	label = df.values[:,b-1]
	dimension = np.shape(data)[1] #solution dimension
	#---------------------------------------------------------------------

	cross = 5
	test_size = (1/cross)
	trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)


	clf=RandomForestClassifier(n_estimators=10)
	clf.fit(trainX,trainy)
	val=clf.score(testX,testy)
	whole_accuracy = val
	#print("Total Acc: ",val)

	x_axis = []
	y_axis = []
	population = initialize(popSize,dimension)
	GBESTSOL = np.zeros(np.shape(population[0]))
	GBESTFIT = 1000

	start_time = datetime.now()

	for currIter in range(1,maxIter):
		newpop = np.zeros((popSize,dimension))
		fitList = allfit(population,trainX,testX,trainy,testy)
		y_axis.append(min(fitList))
		x_axis.append(currIter)
		bestInx = np.argmin(fitList)
		fitBest = min(fitList)
		Xbest = population[bestInx].copy()

		if fitBest<GBESTFIT:
			GBESTFIT = fitBest
			GBESTSOL = Xbest.copy()
			#print("",GBESTSOL.sum())

		for i in range(popSize):
			currFit = fitList[i]
			# print(currFit)
			difference = ( currFit - fitBest ) / currFit
			if difference == 0:
				random.seed(time.time())
				difference = random.uniform(0,1)
			newpop[i] = np.add(population[i],np.multiply(difference,population[i]))
			newpop[i] = toBinary(population[i],dimension,trainX,testX,trainy,testy)

		population = newpop.copy()
	# pyplot.plot(x_axis,y_axis)
	# pyplot.show()

	#test accuracy
	cols = np.flatnonzero(GBESTSOL)
	val = 1
	if np.shape(cols)[0]==0:
		return GBESTSOL
	clf = RandomForestClassifier(n_estimators=10)
	train_data = trainX[:,cols]
	test_data = testX[:,cols]
	clf.fit(train_data,trainy)
	val = clf.score(test_data,testy)
	return GBESTSOL,val


#========================================================================================================
popSize = 20
maxIter = 10
omega = 0.9
datasetList = ["FinalData_3000"]
# datasetList = ["Breastcancer", "BreastEW", "CongressEW", "Exactly", "Exactly2", "HeartEW", "Ionosphere", #"KrVsKpEW","Lymphography", "M-of-n", "PenglungEW", "Sonar", "SpectEW", "Tic-tac-toe", "Vote", "WaveformEW", "Wine", "Zoo"]

for dataset in datasetList:
	accuList = []
	featList = []
	for count in range(3001):
		if (dataset == "FinalData_3000") and count>1500:
			break
		#print(count)
		answer,testAcc = socialmimic("C:/Users/IYI/Desktop/matlab_yedek/aaa_mineral/minerals/birlestirilmis_3000/"+dataset+".csv",popSize,maxIter)
		print(answer.sum())
		accuList.append(testAcc)
		#print(featList.append(answer.sum()))
	inx = np.argmax(accuList)
	best_accuracy = accuList[inx]
	best_no_features = featList[inx]
	print(dataset,"best:",accuList[inx],featList[inx])

	with open("C:/Users/IYI/Desktop/matlab_yedek/aaa_mineral/minerals/birlestirilmis_3000/result_FinalData2.csv","a") as f:
		print(dataset,"%.2f" % (100*best_accuracy),best_no_features,file=f)

1848.0
629.0
764.0
1185.0
981.0
396.0
1192.0
1820.0
1588.0
836.0
1072.0
1065.0
1000.0
1090.0
1031.0
1586.0
1037.0
493.0
1179.0
1758.0
1136.0
1719.0
987.0
802.0
1558.0
533.0
1120.0
1769.0
818.0
740.0
785.0
1364.0
681.0
583.0
1152.0
936.0
1746.0
570.0
888.0
533.0
657.0
1572.0
2068.0
1139.0
1041.0
1914.0
350.0
365.0
972.0
371.0
195.0
1723.0
1008.0
910.0
1365.0
337.0
926.0
279.0
470.0
602.0
579.0
580.0
1342.0
1542.0
671.0
973.0
1077.0
1438.0
518.0
659.0
708.0
1282.0
524.0
1253.0
1285.0
1757.0
1804.0
414.0
974.0
1323.0
649.0
1208.0
1375.0
937.0
633.0
955.0
1830.0
1240.0
525.0
253.0
993.0
1386.0
1319.0
420.0
1240.0
1468.0
2336.0
735.0
2282.0
729.0
526.0
516.0
495.0
1846.0
1202.0
368.0
1627.0
292.0
447.0
715.0
929.0
1124.0
966.0
694.0
1879.0
795.0
1080.0
550.0
533.0
755.0
1870.0
1737.0
630.0
234.0
589.0
802.0
1741.0
1079.0
998.0
1046.0
1033.0
354.0
1310.0
805.0
1330.0
556.0
672.0
179.0
1201.0
730.0
551.0
449.0
897.0
1124.0
882.0
402.0
852.0
382.0
347.0
197.0
968.0
861.0
1078.0
1297.0
707.0
12

471.0
1592.0
599.0
1201.0
1055.0
634.0
680.0
1090.0
329.0
381.0
1637.0
841.0
1675.0
1341.0
486.0
658.0
833.0
624.0
1325.0
392.0
874.0
1005.0
408.0
1106.0
685.0
915.0
1459.0
680.0
529.0
715.0
1394.0
1023.0
472.0
1544.0
733.0
802.0
317.0
1170.0
997.0
537.0
953.0
458.0
1220.0
2200.0
869.0
1385.0
302.0
396.0
875.0
763.0
750.0
934.0
579.0
984.0
386.0
938.0
961.0
1666.0
1244.0
941.0
967.0
516.0
1736.0
510.0
1733.0
413.0
500.0
449.0
1561.0
1449.0
1010.0
963.0
596.0
1269.0
360.0
616.0
477.0
276.0
1865.0
1622.0
1202.0
531.0
339.0
772.0
1227.0
942.0
500.0
863.0
586.0
1036.0
795.0
1204.0
1565.0
1528.0
837.0
1518.0
1480.0
1120.0
184.0
1140.0
2050.0
1352.0
1384.0
1805.0
715.0
1441.0
191.0
424.0
2275.0
611.0
600.0
1024.0
352.0
201.0
2125.0
636.0
704.0
183.0
674.0
519.0
880.0
912.0
1411.0
1181.0
1310.0
356.0
1074.0
208.0
414.0
750.0
1306.0
304.0
1178.0
491.0
2390.0
980.0
938.0
1314.0
1177.0
283.0
424.0
431.0
421.0
576.0
981.0
854.0
956.0
738.0
684.0
830.0
692.0
285.0
675.0
1465.0
260.0
919.0
1408.0
6

IndexError: list index out of range